Author: Ronny F. Pacheco Date: Nov 2024 Copyright: © 2024 Ronny Pacheco License: MIT License

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


# In/Out files

- In files:
    -  2.ingi_in_major/data/sider_df_v2.csv
- Out files:
    - data/blastn_dic (directory) 
    - 2.ingi_in_major/data/bedops/sider.bed
    - 2.ingi_in_major/data/sider_merged.csv 

# Needed modules

In [13]:
import pandas as pd
import pickle
import os
import subprocess

In [ ]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore


@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [ ]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    notebook_name = '1.prepare_data'
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            dict_variables = { # 'var_name' : 'real_var'

            }
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as pickle_file:
            variables = pickle.load(pickle_file)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            print(f"* Loading variable: {pickle_key}")
            globals()[pickle_key] = pickle_value
# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="load")

# Load data

In [4]:
# Load sider_df data
sider_df = pd.read_csv('./data/sider_df_v2.csv', sep=',', header=0)
print(sider_df.shape)
print(sider_df.dtypes)
sider_df.head()

(4804, 6)
chrom           object
length           int64
sider_start      int64
sider_end        int64
sider_strand    object
sider_seq       object
dtype: object


,chrom,length,sider_start,sider_end,sider_strand,sider_seq
0,LmjF.01,253,24284,24536,minus,CCCTCCCCGTGGGCCAGTGTGAGGAGCCGGCTGCGATGCGCTCGAG...
1,LmjF.01,597,24604,25200,minus,TATGACAGTTTGCCGCCCAGCGGCGCTGCGGTGTACCGCTGGCCAG...
2,LmjF.01,935,35467,36401,minus,GCCACGAATTCTTCGTGGGCCTCTCCGTCTCTCCGTGCCATCTGCT...
3,LmjF.01,812,39730,40541,minus,GCCCCAGCCCCTTGGCTGGCATGGACGGAGATGGACGATGAAGACA...
4,LmjF.01,949,54594,55542,minus,GCAGCGGATTTCCATAGGCTTCTCTCAGCTCCAGGCAATCGGTTGA...


# Prepare data

In [7]:
# Take only the needed columns
sider_df = sider_df[['chrom', 'sider_start', 'sider_end']].copy()
sider_df.head()

,chrom,sider_start,sider_end
0,LmjF.01,24284,24536
1,LmjF.01,24604,25200
2,LmjF.01,35467,36401
3,LmjF.01,39730,40541
4,LmjF.01,54594,55542


# Needed functions

In [18]:
# Now let's get the sequences
def get_data_sequence(data, strand, genome_fasta):
    """
    :param data: DataFrame containing information about chromosomal regions including columns 'chrom', 'sider_start', and 'sider_end'.
    :param strand: Specifies the DNA strand to retrieve the sequence from, either 'plus' or 'minus'.
    :param genome_fasta: Path to the genome FASTA file to retrieve sequences from.
    :return: DataFrame with columns 'chrom', 'sider_start', 'sider_end', 'strand', and 'sider_seq' containing the sequences extracted from the genome.
    """
    sequences = []
    for _, row in data.iterrows():
        chrom = row["chrom"]
        start = row["sider_start"]
        end = row["sider_end"]
        blast_cmd = f"blastdbcmd -db {genome_fasta} -entry {chrom} -range {start}-{end} -strand {strand} -outfmt %s"

        sequence = subprocess.check_output(blast_cmd, shell=True, universal_newlines=True).replace('\n', '')

        sequences.append({
            "chrom": chrom,
            "sider_start": start,
            "sider_end": end,
            "strand": strand,
            "sider_seq": sequence
        })

    sequences_df = pd.DataFrame(sequences)
    return sequences_df

# Prepare dict creation
def blastn_dic(path_input, path_output):
    """
    :param path_input: Path to the input file containing nucleotide sequences.
    :param path_output: Path to the output file where the BLAST database will be created.
    :return: None
    """
    # "parse_seqids" is used to keep the sequence ID in the output.
    blast_cmd = f"makeblastdb -in {path_input} -dbtype nucl -parse_seqids -out {path_output}"
    subprocess.run(blast_cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# BEDOPS join strands

## Create bedops folder environment

In [9]:
# Create bedops folder
bedops_folder = './data/bedops'
os.makedirs(bedops_folder, exist_ok=True)

# save bedops sider file
sider_bedops_path = './data/bedops/sider.bed'
sider_df.to_csv(sider_bedops_path, sep='\t', header=False, index=False)

## Bedops merge

In [11]:
# Bedops merge call
cmd = f"bedops --merge {sider_bedops_path}"
sider_bedops = subprocess.check_output(cmd, shell=True, universal_newlines=True)
sider_bedops = pd.DataFrame([x.split("\t") for x in sider_bedops.split("\n") if x],
                              columns=["chrom", "sider_start", "sider_end"])
sider_bedops[["sider_start", "sider_end"]] = sider_bedops[["sider_start", "sider_end"]].apply(pd.to_numeric)
print(f"From a total of {sider_df.shape[0]} there are {sider_bedops.shape[0]} SIDER sequences after BEDOPS merge")

From a total of 4804 there are 2874 SIDER sequences after BEDOPS merge


In [12]:
# Prepare dictionary blastn
blast_dic_path = "../data/blastn_dic"
os.makedirs(blast_dic_path, exist_ok=True)

## Blastn dict preparation

In [16]:
genome_path = "../data/TriTrypDB-68_LmajorFriedlin_Genome.fasta"
blastn_dict_output_path = "../data/blastn_dic/TriTrypDB-68_LmajorFriedlin_Genome.fasta"
blastn_dic(path_input=genome_path,
           path_output=blastn_dict_output_path)

In [21]:
sider_merged = get_data_sequence(sider_bedops, "plus", blastn_dict_output_path)
print(sider_merged.shape)
print(sider_merged.dtypes)
sider_merged.head()

(2874, 5)
chrom          object
sider_start     int64
sider_end       int64
strand         object
sider_seq      object
dtype: object


,chrom,sider_start,sider_end,strand,sider_seq
0,LmjF.01,24284,24536,plus,GCAGCCCCCTCCCCCTCTATCCCTGCCAACGCCAAACCACTTCTGG...
1,LmjF.01,24604,25200,plus,CCGGCATGATGGAGGGGCGGCTGTGAGGTGGTGCGTGCATGTATGT...
2,LmjF.01,35467,36401,plus,CCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACACGCGCTGCAG...
3,LmjF.01,39730,40541,plus,ATTTTACCGCGAGCAAGGCAACACACAGACGCACACACAACCACAG...
4,LmjF.01,54594,55542,plus,GCCGCACCCCACCCCCATCGCCTGGTGCGAAGCAGCGCAAGACACA...


# Save data

In [22]:
file_path = "./data/sider_merged.csv"
sider_merged.to_csv(file_path, sep=',', header=True, index=False)